## **Setting Up Bing Integration (Tool) with Azure AI Agents**

### **📝 Overview**

Grounding with Bing Search enables your Azure AI Agents to incorporate real-time public web data when generating responses. By leveraging Bing Search, your agent can fetch the latest information—such as top news or industry updates—and return relevant, cited content. 

When a user sends a query, the agent first determines if it should invoke Bing Search. If so, it sends a Bing search query (using only your resource key for billing and rate limiting) and retrieves search results. These results are then used to generate a final, human-readable response that includes required citations and links, as specified by Microsoft’s use and display requirements.

> **Important**: Usage of Grounding with Bing Search can incur additional costs. For pricing and legal details, review the [Bing Grounding Terms](https://www.microsoft.com).


### **✅ Prerequisites**

1. **Grounding with Bing Search Resource**
   - Create a Grounding with Bing Search resource using the Azure portal or a code-first approach.
   - Ensure you have **Owner** or **Contributor** permissions to register the resource provider `Microsoft.Bing` if using a code-first approach.

2. **Azure AI Agent Setup**
   - Ensure you have an Azure AI Agent created (via the Agent Playground or quickstart).

3. **Permission/Role Assignment**
   - **Access to Bing Search**: Users do not have access to raw search content; instead, they receive model-generated responses that include citations and links.
   - **RBAC for Foundry Project**: End users must have the `AI Developer` role assigned.

4. **Supported Models**
   - Grounding with Bing Search currently works with specific Azure OpenAI models (e.g., `gpt-3.5-turbo-0125`, `gpt-4-0125-preview`, etc.).

5. **Create a New Connection**
   - Provide the required key-value pairs for the connection:
   - **`resource-key`**: Your Bing resource key.
   - **`endpoint`**: Your Bing search endpoint (typically provided in your resource details).

6. **Add a Knowledge Source**
   - Click to add a knowledge source and select **Grounding with Bing Search**.
   - If you don’t see this option:
      - Verify that your resource has been created in the same resource group as your AI Agent.
      - Ensure you have the necessary permissions.


In [1]:
import os

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Define the target directory
target_directory = os.getcwd()  # Get the current working directory

# Move one directory back
parent_directory = os.path.dirname(target_directory)

# Check if the parent directory exists
if os.path.exists(parent_directory):
    # Change the current working directory to the parent directory
    os.chdir(parent_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Parent directory {parent_directory} does not exist.")

Directory changed to c:\Users\pablosal\Desktop\gbb-ai-agenticrag


In [2]:
import importlib.metadata as md

# Versions - we are currently 1.0.0b9 of azure-ai-projects
print("semantic-kernel version:", md.version("semantic-kernel"))
# if you want to Upgrade the SDKs, uncomment the line below but code might break
# %pip install -U semantic-kernel azure-ai-projects azure-identity

semantic-kernel version: 1.28.1


In [3]:
import os
import re
import time
import logging
import json
from datetime import datetime as pydatetime
from typing import Any, List, Dict, Optional
from dotenv import load_dotenv
import asyncio
from datetime import timedelta

# Azure AI Projects
from azure.identity.aio import DefaultAzureCredential
from azure.core.exceptions import HttpResponseError

# semantic kernel
from semantic_kernel.contents import AuthorRole
from semantic_kernel.agents.azure_ai.azure_ai_agent import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.open_ai.run_polling_options import RunPollingOptions

# Load environment variables from .env file
load_dotenv()

# configure logging
from utils.ml_logging import get_logger

logger = get_logger()

In [ ]:
from azure.core.exceptions import ServiceRequestError
from azure.ai.projects.aio import AIProjectClient

async def get_connection_id(client: AIProjectClient, env_var: str) -> Optional[str]:
    """
    Retrieves the connection object using a connection name stored in an environment variable.

    Args:
        client: The Azure AI Project client.
        env_var (str): The environment variable holding the connection name.

    Returns:
        Connection object if found, otherwise raises an error.
    """
    connection_name = os.getenv(env_var)
    if not connection_name:
        logger.error(f"Missing environment variable: '{env_var}'")
        raise ValueError(f"Environment variable '{env_var}' is required.")

    try:
        connection = await client.connections.get(connection_name=connection_name)
        logger.info(f"Retrieved Connection ID for {env_var}: {connection.id}")
        return connection
    except Exception as e:
        logger.error(f"Failed to retrieve connection for {env_var}: {e}")
        raise

import json
import logging
from datetime import timedelta

def process_citations(content) -> str:
    """
    Return content plus any citations in Markdown format, ensuring no duplicated citations.

    Args:
        content: The content object retrieved from the agent.

    Returns:
        A string containing the agent's text plus any unique citations in Markdown format.
    """
    combined_content = content.content

    if hasattr(content, "items"):
        unique_citation_set = set()
        for item in content.items:
            if item.content_type == "annotation" and item.url:
                unique_citation_set.add((item.quote, item.url))

        if unique_citation_set:
            combined_content += "\n\n**Citations:**\n"
            for quote, url in unique_citation_set:
                combined_content += f"- **Quote**: {quote}  \n"
                combined_content += f"  **URL**: [{url}]({url})\n"

    return combined_content

async def run_agent(
    project_client,
    agent_id: str,
    user_input: str
) -> str:
    """
    Runs the data-retrieval agent to process a single user input and returns
    a single string containing both user input and the agent responses.

    Args:
        project_client: A client object for interacting with Azure AI project resources.
        agent_id: The ID of the agent to retrieve from project_client.
        user_input: A string representing the user's query.

    Returns:
        A single string with user input and each agent response (enriched with citations).
    """
    # Initialize the string to include the user query
    responses = f"👤 User: {user_input}\n"

    # Fetch the agent definition using the provided ID
    agent_definition = await project_client.agents.get_agent(agent_id)
    
    # Create the AzureAIAgent instance
    agent = AzureAIAgent(
        client=project_client,
        definition=agent_definition,
        polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Create a new conversation thread
    thread = await project_client.agents.create_thread()

    try:
        # Send user input to the agent
        await agent.add_chat_message(thread_id=thread.id, message=user_input)
        
        # Stream non-tool responses from the agent
        async for content in agent.invoke(thread_id=thread.id):
            enriched_content = process_citations(content)
            responses += f"\n🤖 {agent.name}: {enriched_content}"

    except HttpResponseError as e:
        try:
            error_json = json.loads(e.response.content)
            message = error_json.get("Message", "No error message found")
            logging.error(f"❌ **HTTP Error:** {message}")
            responses += f"\n❌ **HTTP Error:** {message}"
        except json.JSONDecodeError:
            logging.error(f"❌ **Non-JSON Error Content:** {e.response.content}")
            responses += f"\n❌ **Non-JSON Error Content:** {e.response.content}"

    return responses, thread.id

### **Create Client and Load Azure AI Foundry**

Here, we initialize the Azure AI client using DefaultAzureCredential. This allows us to authenticate and connect to the Azure AI service.

In [5]:
project_client = AzureAIAgent.create_client(credential=DefaultAzureCredential(),
conn_str=os.getenv("AZURE_AI_FOUNDRY_CONNECTION_STRING"))

In [ ]:
from azure.ai.projects.models import (
    BingGroundingTool,
)

# Initialize Azure AI Agent settings
validationinsightagent_settings = AzureAIAgentSettings.create()

# Create a ToolSet to manage tools
toolset = ToolSet()

try:
    # Retrieve and add the Bing Grounding Tool
    bing_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_BING")
    toolset.add(BingGroundingTool(connection_id=bing_connection.id))
    logger.info("Bing Grounding Tool added successfully.")
 
    logger.info("Successfully created ToolSet with Bing and File Search tools.")
except Exception as e:
    logger.error(f"Failed to create ToolSet: {e}")
    raise

The create method is deprecated. Use the __new__ method instead.


In [ ]:
dataretrievalagent_settings_definition = await project_client.agents.create_agent(
    model=dataretrievalagent_settings.model_deployment_name,
    name="BingDataRetrievalAgent",
    description=(
        "An AI agent specialized in retrieving and analyzing real-time data from the web using Bing. "
        "This includes retrieving up-to-date information, news, research articles, and other publicly available web content. "
        "The agent is designed to assist in research and decision-making by providing accurate, relevant, and actionable insights. "
        "If no relevant data is found, the agent must clearly indicate this and provide suggestions for refining the query."
    ),
    instructions=(
        "### Role & Objective\n"
        "You are a research-focused AI assistant responsible for retrieving and analyzing real-time data exclusively from the web using Bing. "
        "Your goal is to provide precise, well-referenced, and relevant responses to support research, decision-making, and analysis efforts.\n\n"
        
        "### Data Retrieval & Prioritization\n"
        "1. **Real-Time Web Data (Bing):** \n"
        "   - Use Bing to retrieve real-time data, including news, research articles, and other publicly available web content.\n"
        "   - Focus on extracting key insights, summaries, and actionable information from the retrieved web pages.\n"
        "   - Example: 'Retrieve the latest news on advancements in AI technology,' or 'Find research articles on the impact of climate change on agriculture.'\n\n"
        
        "2. **Source Evaluation:** \n"
        "   - Prioritize credible and authoritative sources (e.g., academic journals, government websites, reputable news outlets).\n"
        "   - Avoid using unreliable or unverified sources.\n\n"
        
        "3. **Integrated Queries:** \n"
        "   - If the query requires multiple pieces of information, retrieve and integrate the results for a comprehensive response.\n"
        "   - Ensure clarity in presenting combined insights.\n\n"
        
        "4. **Fallback Behavior:** \n"
        "   - If no relevant data is found on the web, respond with:\n"
        "     - A clear statement that no relevant data was found.\n"
        "     - Suggestions for refining the query or exploring alternative approaches.\n"
        "     - Example: 'No relevant data was found for the requested query on the web. Consider refining your query or specifying additional details.'\n\n"
        
        "### Response Quality\n"
        "1. **Accuracy & Relevance:** Always prioritize retrieving the most current and applicable data from credible sources.\n"
        "2. **Clarity & Transparency:** Clearly indicate the data source(s) used and any limitations in the available information.\n"
        "3. **Fallback Handling:** If no relevant data is found, provide a professional and helpful fallback response as outlined above.\n"
        "4. **Professionalism:** Present findings in a structured and concise manner to facilitate decision-making.\n"
        "5. **Source Context:** Ensure that extracted insights are presented with sufficient context to maintain their relevance and accuracy.\n"
    ),
    toolset=toolset,
    headers={"x-ms-enable-preview": "true"},
    temperature=0.7,
    top_p=1,
    metadata={
        "use_case": "Real-Time Web Data Retrieval for Research and Decision-Making",
        "data_source": "Bing (web search)",
    },
)

# Print the agent's run ID (agent ID)
print(f"BingDataRetrievalAgent Run ID: {dataretrievalagent_settings_definition.id}")

In [ ]:
user_query = "In which glucose ranges does Product A underperform compared to Product B, and what clinical impact could this have?"
fabric_response, threadID = await run_agent(project_client, "asst_iIMaKxWz4JUJ5YyWqoSR9t3n", user_query)